# Developing a Model to Analyze Squad Battles Selection

## Environment Setup

In [16]:
%%capture
%pip install huggingface_hub numpy python-dotenv ultralytics

## Common Utilities

This section includes utility functions that are common across various parts of the notebook, aimed at streamlining workflow and simplifying code reuse.

### Additional helper functions can be defined below as needed

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file located one directory above the current notebook directory
load_dotenv(os.path.abspath(os.path.join('..', '.env')))  # Use relative path

# Retrieve the Hugging Face username and token from the environment
hf_username = os.getenv("HF_USERNAME")
hf_token = os.getenv("HF_TOKEN")

## Training
Train a model to classify available options in the squad battles selection menu.  

In [ ]:
from ultralytics import YOLO
import os

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use YOLOv8n, v8s, etc., depending on your needs

# Define the dataset path
dataset_path = os.path.expanduser("~/Downloads/project-3-at-2024-12-07-19-47-2f045ff7/dataset.yaml")  # Use os.path.expanduser to handle '~'

# Train model
model.train(
    data=dataset_path,  # Updated path to the dataset YAML
    imgsz=640,
    epochs=50,
    batch=16,
    name="yolo_training",
)

# Inference of the Model

In [ ]:
from dotenv import load_dotenv
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from PIL import Image  # Ensure correct import for Image
import numpy as np

import sys
import os

# Adding the directory containing 'src' to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Load src.config to check the variables
import src.config as config

# Load the trained YOLO model
model_path = hf_hub_download(config.HF_SQUAD_SELECTION_PATH, config.SQUAD_SELECTION_FILENAME)
model = YOLO(model_path)

# Directory containing the images for inference
source_dir = os.path.expanduser("../screenshots/jpg/cropped")  # Expand ~ to full path
output_dir = "runs/detect/inference"  # YOLO saves results here by default

# Iterate through all images in this directory
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"): 
        image_path = os.path.join(source_dir, filename)
        image = Image.open(image_path)

        # Send the image to the YOLO model for detection
        # This will output to stdout by default
        result = model.predict(source=image, imgsz=640, conf=0.25)
        if len(result[0].boxes) > 0:  # Check if there are no boxes
            print(result[0].boxes)
            # From here, the bbox coordinates can be retrieved for a deeper analysis

# Upload to HuggingFace

In [ ]:
from huggingface_hub import HfApi, Repository
import os
import shutil

# Set your Hugging Face token and repository name
repo_name = "fc24-squad-battles-selection"  # Replace with the desired repository name

# Define paths
model_dir = "./runs/detect/yolo_training" 
local_repo_dir = f"huggingface_yolo_repo"  # Temporary local directory for the repository

# Create a new repository on Hugging Face (or use an existing one)
api = HfApi()
repo_url = api.create_repo(repo_id=repo_name, token=hf_token, exist_ok=True)

# Clone the repository to a local folder with the token
repo = Repository(local_dir=local_repo_dir, clone_from=repo_url, use_auth_token=True)

# Copy all files from the training directory to the repository
shutil.copytree(model_dir, os.path.join(local_repo_dir, "weights"), dirs_exist_ok=True)

# Push the files to Hugging Face
repo.push_to_hub(commit_message="Upload YOLO training output")

print(f"Files uploaded successfully! View the repository here: {repo_url}")
